In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm
sm.set_framework('tf.keras')
import multiprocessing as mp
from save_weights_every_epoch import CallbackForSavingModelWeights
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import multiprocessing as mp
import dask.array as da

Segmentation Models: using `keras` framework.


In [2]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [3]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [4]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../ml-data-training/ship_segmentation_data/train_v2/6384c3e78.jpg']

In [5]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [6]:
train, val, test = split_datasets(csv_file)

In [7]:
len(train)

188722

In [8]:
(256 * 256) * 188722

12368084992

In [9]:
def get_arr(data, shape = (256, 256)):
    mask = tf.io.read_file(data)
    mask = tf.image.decode_jpeg(mask, channels=1)
    mask = tf.image.resize(mask, size=shape)
    mask = mask / 255
    mask = mask.numpy()
    uniques = np.unique(mask, return_counts = True)
    return uniques

def handler_class_weights(data_arr):
    zeros, ones = 0, 0
    for x in tqdm(range(len(data_arr))):
        uniques = get_arr(data_arr[x])
        for y in range(len(uniques[0])):
            if uniques[0][y] == 0.0:
                zeros += uniques[1][y]
            elif uniques[0][y] == 1.0:
                ones += uniques[1][y]
    return [zeros, ones]

In [10]:
# returns = handler_class_weights(train['mask_paths'].values.tolist())

In [11]:
# returns
[12354158995, 13925997]

[12354158995, 13925997]

In [14]:
w0 = (12354158995 + 13925997) / (2 * 12354158995)
w1 = (12354158995 + 13925997) / (2 * 13925997)

In [16]:
w0

0.5005636157429104

In [17]:
w1

444.0646149787337

In [9]:
from sklearn.utils import compute_class_weight

In [15]:
12354158995 + 13925997

12368084992

In [10]:
zero = np.zeros(shape=(12354158995 + 13925997))

In [11]:
zero[12354158996:] = 1

In [12]:
compute_class_weight(class_weight='balanced', classes=[0,1], y=zero)